# Get the Data

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
import warnings
warnings.filterwarnings("ignore")

DATASET2_ODBC = 'PROTO.Melih.Dataset2_SimilarProducts'
DATASET2_TITLE = 'dataset2'

In [3]:
from common.sql_helper import custom_query

query = f'select * from {DATASET2_ODBC}'
result = custom_query(query=query)

result.head()

,WebSecenek,MarkaKodu,CinsiyetKodu,UrunGrubuKodu,Renk,cUrl,UrlNo
0,50189274-VR046,08,1,016,VR046,https://resim.aydinli.com.tr/08/1/016/S5018927...,2
1,50189277-VR033,08,1,016,VR033,https://resim.aydinli.com.tr/08/1/016/S5018927...,1
2,50189277-VR033,08,1,016,VR033,https://resim.aydinli.com.tr/08/1/016/S5018927...,2
3,50189278-VR085,08,1,016,VR085,https://resim.aydinli.com.tr/08/1/016/S5018927...,1
4,50189278-VR085,08,1,016,VR085,https://resim.aydinli.com.tr/08/1/016/S5018927...,2


In [4]:
result.shape

(125246, 7)

In [5]:
class_names = result[['CinsiyetKodu', 'UrunGrubuKodu']].drop_duplicates()
class_names.head()

,CinsiyetKodu,UrunGrubuKodu
0,1,016
63,2,016
83,2,057
93,2,013
111,1,013


In [6]:
class_names.shape

(50, 2)

In [ ]:
# from common.model_helper import make_dataset_cv

# errors = make_dataset_cv('08', 'DATASETDENEME', result,class_names , 1000)
# errors

# Creating Model

## Creating Model CheckPoint

In [3]:
checkpoint_path = "checkpoints/TL_training_classifier/checkpoint.ckpt" # note: remember saving directly to Colab is temporary
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                         save_weights_only=True,
                                                         save_best_only=True,
                                                         verbose=1)

## Flow Data

In [4]:
dir_ds_2 =  '../datasets/dataset2/brand_08/'

In [5]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.2)

train_generator = datagen.flow_from_directory(dir_ds_2,
                                              target_size=(224, 224),
                                              batch_size=32,
                                              class_mode='categorical',
                                              subset="training",
                                              shuffle=True) # training dataset
test_generator = datagen.flow_from_directory(dir_ds_2,
                                             target_size=(224, 224),
                                             batch_size=32,
                                             class_mode="categorical",
                                             subset="validation",
                                             shuffle=True
                                             ) # validation dataset

Found 97761 images belonging to 50 classes.
Found 24415 images belonging to 50 classes.


In [8]:
train_generator.num_classes

50

## Model 0: Building a transfer learning model using the Keras Functional API

In [9]:
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

inputs = tf.keras.layers.Input(shape=(224, 224, 3), name="input_layer")
x = base_model(inputs)
x = tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling2D")(x)
outputs = tf.keras.layers.Dense(train_generator.num_classes, name="output_layer")(x)

model_0 = tf.keras.Model(inputs, outputs)

In [11]:
model_0.compile(loss="categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy']
                )

history_0 = model_0.fit(train_generator,
                        epochs=5,
                        steps_per_epoch=len(train_generator),
                        validation_data=test_generator,
                        validation_steps=int(0.25 * len(test_generator)),
                        callbacks=[checkpoint_callback])

Epoch 1/5
3056/3056 [==============================] - ETA: 0s - loss: 9.1264 - accuracy: 0.2342
Epoch 1: val_loss improved from inf to 8.99029, saving model to checkpoints/TL_training_classifier\checkpoint.ckpt
3056/3056 [==============================] - 4164s 1s/step - loss: 9.1264 - accuracy: 0.2342 - val_loss: 8.9903 - val_accuracy: 0.2602
Epoch 2/5
3056/3056 [==============================] - ETA: 0s - loss: 9.0197 - accuracy: 0.2826
Epoch 2: val_loss did not improve from 8.99029
3056/3056 [==============================] - 4060s 1s/step - loss: 9.0197 - accuracy: 0.2826 - val_loss: 9.4349 - val_accuracy: 0.2896
Epoch 3/5
3056/3056 [==============================] - ETA: 0s - loss: 9.0109 - accuracy: 0.3019
Epoch 3: val_loss did not improve from 8.99029
3056/3056 [==============================] - 4084s 1s/step - loss: 9.0109 - accuracy: 0.3019 - val_loss: 9.5500 - val_accuracy: 0.3109
Epoch 4/5
3056/3056 [==============================] - ETA: 0s - loss: 8.8732 - accuracy: 0.322

## Evaluation

The model_0 has been trained in remote desktop. So we need to copy the architecture. 

In [18]:
model_0

In [ ]:
# coming back 